In [4]:
import clickhouse_connect

client = clickhouse_connect.get_client(host='dd8vv8kg7q.us-east-1.aws.clickhouse.cloud', port=8443, username='default', password='C4.Z.~~oGqX5p')

In [41]:
import uuid


In [42]:
import pandas as pd

df = pd.read_excel('aircraft_parts_data.xlsx', sheet_name='Dataset')
# df = pd.read_sql('SELECT * FROM default.test', client)
df['airId'] = df.apply(lambda x: uuid.uuid4(), axis=1)

In [43]:
for i in df.columns:
    if df[i].dtype == 'object':
        df[i] = df[i].astype('string')

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 29 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Part Name                                  10000 non-null  string 
 1   Material Composition                       10000 non-null  string 
 2   Age (years)                                10000 non-null  float64
 3   Condition                                  10000 non-null  string 
 4   Location                                   10000 non-null  string 
 5   Manufacturer                               10000 non-null  string 
 6   Aircraft Model                             10000 non-null  string 
 7   Potential Use Cases                        10000 non-null  string 
 8   New Parts Carbon Footprint (kg CO2e)       10000 non-null  float64
 9   Recycled Parts Carbon Footprint (kg CO2e)  10000 non-null  float64
 10  Water Usage - New Parts

In [68]:
# Get the column names and data types of the DataFrame
columns = df.columns
data_types = df.dtypes

print(data_types.tolist())


# Insert the DataFrame into the ClickHouse table


[string[python], string[python], dtype('float64'), string[python], string[python], string[python], string[python], string[python], dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), string[python]]


In [64]:
statement = """CREATE TABLE {} (
    {},
    PRIMARY KEY (airId)
  );""".format(
      'dev.airb',
      ', '.join('"{}" {}'.format(column, str(data_type).capitalize()) for column, data_type in zip(columns, data_types))
      
  )
statement

'CREATE TABLE dev.airb (\n    "Part Name" String, "Material Composition" String, "Age (years)" Float64, "Condition" String, "Location" String, "Manufacturer" String, "Aircraft Model" String, "Potential Use Cases" String, "New Parts Carbon Footprint (kg CO2e)" Float64, "Recycled Parts Carbon Footprint (kg CO2e)" Float64, "Water Usage - New Parts (liters)" Float64, "Water Usage - Recycled Parts (liters)" Float64, "Landfill Waste - New Parts (kg)" Float64, "Landfill Waste - Recycled Parts (kg)" Float64, "Energy Consumption - New Parts (kWh)" Float64, "Energy Consumption - Recycled Parts (kWh)" Float64, "Recycling Rate (%)" Float64, "Toxicity Score - New Parts" Float64, "Toxicity Score - Recycled Parts" Float64, "Remanufacturing Potential" Float64, "Life Cycle Assessment" Float64, "Renewable Material Content (%)" Float64, "Carbon Footprint Saved (kg CO2e)" Float64, "Water Usage Saved (liters)" Float64, "Landfill Waste Saved (kg)" Float64, "Energy Consumption Saved (kWh)" Float64, "Toxicity

In [65]:
client.command(statement)

''

In [67]:
client.insert_df('airb', df,database='dev')